# Stream and View an NWB File with fsspec

### Environment Setup

In [ ]:
!pip install dandi
!pip install fsspec requests aiohttp
!pip install nwbwidgets

### Download files
Run these cells for each dandiset and filepath you want to download.
No need to run these if the file you're interested in is already downloaded.

In [ ]:
### download configuration

dandiset_id = "000336"
# file path within dandiset (don't include leading '/')
filepath = "sub-1202533446/sub-1202533446_ophys.nwb"

# if you want to access an embargoed dandiset, you must set 'authenticate' to True and provide a dandi api key
# your dandi api key can be found on dandi, if you press your profile icon in the top-right corner
authenticate = True
dandi_api_key = "c3ccc16df82a86fd8c25a6b233bfd27d9da3f2b4"


In [ ]:
from dandi import dandiapi
import dandi

if authenticate:
    client = dandiapi.DandiAPIClient(token=dandi_api_key)
else:
    client = dandiapi.DandiAPIClient()
    
my_dandiset = client.get_dandiset(dandiset_id)


In [ ]:
file = my_dandiset.get_asset_by_path(filepath)
file_url = file.get_content_url(follow_redirects=1, strip_query=True)

In [ ]:
io = pynwb.NWBHDF5IO(file=file, mode='r', load_namespaces=True)
nwb = io.read()
nwb

### Display nwb file
Choose the nwb file you want to view and display it here!

In [ ]:
import fsspec
import pynwb
import h5py
from fsspec.implementations.cached import CachingFileSystem

# first, create a virtual filesystem based on the http protocol and use
# caching to save accessed data to RAM.
fs = CachingFileSystem(
    fs=fsspec.filesystem("http"),
    cache_storage="nwb-cache",  # Local folder for the cache
)

# next, open the file
with fs.open(file_url, "rb") as f:
    with h5py.File(f) as file:
        with pynwb.NWBHDF5IO(file=file, mode='r', load_namespaces=True) as io:
            nwb = io.read()

In [ ]:
### uncomment these to view aspects of the file
### note that not all these properties exist for all nwb files
print(nwb.electrodes)
# nwb.identifier
# nwb.processing
# nwb.acquisition["events"]
# nwb.intervals["trials"]
# nwb.stimulus["StimulusPresentation"]
# nwb.electrodes

In [ ]:
nwb